# Algoritmi supervisionati

## In questa sezione vengono applicati gli algoritmi supervisionati

### Import delle librerie e moduli necessari

In [57]:
# librerie generali
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# librerie per la rappresentazione del dataset
from sklearn.feature_extraction.text import TfidfVectorizer

# libreria per suddividere il dataset in train e test set
from sklearn.model_selection import train_test_split

# libreria utilizzata per l'ottimizzazione dei parametri
from sklearn.model_selection import GridSearchCV

# librerie utilizzate per le misure
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

# libreria utilizzata per la cross validation
from sklearn.model_selection import cross_val_score

# libreria per il Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# libreria utilizzata per il Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# libreria utilizzata per il KNN
from sklearn.neighbors import KNeighborsClassifier

# libreria utilizzata per la rete neurale
from sklearn.neural_network import MLPClassifier

# libreria utilizzata per il support vector machine
from sklearn.svm import SVC

# seed utilizzato per la generazione del train e test set
random_seed = 3

# 1. Creazione dataset con rappresentazione tf-idf

In [58]:
%store -r clean_dataset

In [59]:
def createDataframeWithTfIdf(dataframe):
    reviewText = dataframe['reviewText'].to_numpy()
    summary = dataframe['summary'].to_numpy()
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    tfidf_vector = vectorizer.fit_transform(reviewText + summary)
    tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=[reviewText, summary], columns=vectorizer.get_feature_names())
    
    tfidf_df['numberPositiveReview'] = dataframe['numberPositiveReview'].to_numpy()
    tfidf_df['numberNegativeReview'] = dataframe['numberNegativeReview'].to_numpy()
    tfidf_df.reset_index(drop=True, inplace=True)
    
    tfidf_df['numberPositiveSummary'] = dataframe['numberPositiveSummary'].to_numpy()
    tfidf_df['numberNegativeSummary'] = dataframe['numberNegativeSummary'].to_numpy()
    tfidf_df.reset_index(drop=True, inplace=True)
    
    sentiment = dataframe['sentiment']
    return tfidf_df, sentiment

In [60]:
dataset, labels = createDataframeWithTfIdf(clean_dataset)

In [61]:
print(dataset.shape)

(12000, 51175)


# 2. Definizione della funzione K-fold cross validation

## Questa funzione viene utilizzata per effettuare la K-fold cross validation sui varia classificatori. La cross validation viene effettuata per valori di k in [10, 15]

In [62]:
def kCrossValidation(model, dataset, labels):
    k = []
    acc = []
    dev = []

    #nota, non viene effettuato lo shuffle dei fold, quindi sono sempre gli stessi, sono già istanziati
    for i in range(10, 16):
        scores = cross_val_score(model, dataset, labels, cv=i)
        print("K cross validation, k= ", i)
        k.append(i)
        print("Average scores: ", scores.mean())
        acc.append(scores.mean())
        print("Standard Deviation of scores: ", scores.std())
        dev.append(scores.std())
        print("\n\n")

# 3. Random Forest

## 3.1 Random Forest: Feature Selection

### Prima di applicare il random forest per la classificazione, viene effettuata la feature selection per estrarre le feature più importanti, avendo il dataset nella rappresentazione tf-idf

In [65]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset, labels, test_size=0.2, random_state=random_seed)
    
rf = RandomForestClassifier(criterion = 'entropy', random_state=random_seed)
rf.fit(X_train, Y_train)
y_pred=rf.predict(X_test)

# feature selection
sel = SelectFromModel(rf, prefit=True)
selected_feat = dataset.columns[(sel.get_support())]
print("Numero feature selezionate: " + str(len(selected_feat)))

featureSelection_df = dataset.loc[:, sel.get_support()]

MemoryError: Unable to allocate 4.58 GiB for an array with shape (51175, 12000) and data type float64

## 3.2 Random Forest: Parameter Tuning

### Dopo aver effettuato la feature selection, viene attuato il paramater tuning sulle feature più importanti tramite una ricerca esaustiva dello spazio di ricerca

### Parametri su cui viene effettuata la ricerca

In [ ]:
n_estimators_list = [40,60,80,100]
criterion_list = ['entropy']
max_depth_list = [35,55]
max_depth_list.append(None)
min_samples_split_list = [15, 30]
min_samples_leaf_list = [5, 15, 30]
max_features_list = ['log2']

In [ ]:
params_grid = {
    'n_estimators': n_estimators_list,
    'criterion': criterion_list,
    'max_depth': max_depth_list,
    'min_samples_split': min_samples_split_list,
    'min_samples_leaf': min_samples_leaf_list,
    'max_features': max_features_list
}

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(featureSelection_df, labels, test_size=0.2, random_state=random_seed)


def my_acc_score(model, truncated_df, labels):
    return metrics.accuracy_score(labels, model.predict(truncated_df))

### Ricerca dei parametri migliori

In [ ]:
rf_searchParameter = GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced'),
                            param_grid=params_grid,
                            cv=3,
                            return_train_score=True,
                            verbose=2,
                            scoring = my_acc_score
                            )

rf_searchParameter.fit(X_train, Y_train)

### Parametri migliori trovati

In [ ]:
best_param = rf_searchParameter.best_params_
print("Parametri migliori trovati:\n")
for pair in best_param.items():
    print(pair)

In [ ]:
df_cv_results = pd.DataFrame(rf_searchParameter.cv_results_)
df_cv_results = df_cv_results[['rank_test_score','mean_test_score','mean_train_score',
                           'param_n_estimators', 'param_min_samples_split','param_min_samples_leaf',
                           'param_max_features', 'param_max_depth','param_criterion']]
df_cv_results.sort_values('rank_test_score', inplace=True)
df_cv_results[:10]

## 3.3 Random Forest: Classificazione

### Prendo i parametri migliori trovati per effettuare la classificazione

In [ ]:
n_estimators_found = best_param.get('n_estimators')
max_depth_found = best_param.get('max_depth')
max_features_found = best_param.get('max_features')
min_samples_leaf_found = best_param.get('min_samples_leaf')
min_samples_split_found = best_param.get('min_samples_split')
criterion_found = best_param.get('criterion')

### Task di classificazione

In [ ]:
rf = RandomForestClassifier(n_estimators = n_estimators_found, 
                            criterion = criterion_found,
                            class_weight= 'balanced',
                            max_depth= max_depth_found,
                            max_features= max_features_found,
                            min_samples_leaf= min_samples_leaf_found,
                            min_samples_split= min_samples_split_found)
rf.fit(X_train, Y_train)
y_predicted = rf.predict(X_test)

In [ ]:
print("Accuracy score:",accuracy_score(Y_test, y_predicted))
print(classification_report(Y_test, y_predicted, target_names=["Negativo", "Positivo"]))

## K-fold Cross Validation

In [ ]:
kCrossValidation(rf, featureSelection_df, labels)

# 4. Multinomial Naive Bayes

## Task di classificazione

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(featureSelection_df, labels, test_size=0.2, random_state=random_seed)
nb = MultinomialNB()
nb.fit(X_train, Y_train)
y_predicted = nb.predict(X_test)

In [ ]:
print("Accuracy score: ", accuracy_score(Y_test, y_predicted))
print(classification_report(Y_test, y_predicted, target_names=["Negativo", "Positivo"]))
    

## K-fold Cross Validation

In [ ]:
kCrossValidation(nb, featureSelection_df, labels)

# 5. KNN

## 5.1 Trovare il k migliore

### Prima di effettuare la classifcazione con il KNN, si cerca il valore k migliore che identifica il numero di neighbors su cui basarsi per la classficazione

### La ricerca di k viene effettuata per i seguenti valori: [9,11,13,15,17,19,97]

### Funzione che ricerca il k migliore

In [ ]:
def findBestK(dataset, labels):
    X_train, X_test, Y_train, Y_test = train_test_split(dataset, labels, test_size=0.2, random_state=random_seed)
    error = []
    
    acc_train = []
    acc_test = []
    max_acc_test = 0
    max_acc_train = 0
    best_k = 0
    
    root = int(math.sqrt(X_train.shape[0]))
    k = [9,11,13,15,17,19]
    k.append(root)
    
    for i in k:
        print(str(i) + " Neighbors")
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train, Y_train)
        pred_i = knn.predict(X_test)
        
        error.append(np.mean(pred_i != Y_test))
        
        print("train: ", knn.score(X_train, Y_train))
        print("test: ", knn.score(X_test, Y_test))
        acc_train.append(knn.score(X_train, Y_train))
        acc_test.append(knn.score(X_test, Y_test))
        print()
        if knn.score(X_train, Y_train) >= max_acc_train and knn.score(X_test, Y_test) >= max_acc_test:
            best_k = i
            max_acc_test = knn.score(X_test, Y_test)
            max_acc_train = knn.score(X_train, Y_train)
        
    print("\nErrori per i vari k:\n")
    print(error)
    
    x=[9,11,13,15,17,19,97]
    values = range(len(x))

    plt.figure(figsize=(12, 6))
    plt.xticks(values,x)
    plt.plot(values, error, color='red', linestyle='dashed', marker='o',
             markerfacecolor='blue', markersize=10)
    plt.title('Error Rate K Value')
    plt.xlabel('K Value')
    plt.ylabel('Mean Error')

    return best_k

## 5.2 KNN: Classificazione

### Task di classificazione

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(featureSelection_df, labels, test_size=0.2, random_state=random_seed)


In [ ]:
k = findBestK(featureSelection_df, labels)
knc = KNeighborsClassifier(n_neighbors=k, weights="distance")
knc.fit(X_train, Y_train)
y_predicted = knc.predict(X_test)

In [ ]:
print("Accuracy score: ", accuracy_score(Y_test, y_predicted))
print(classification_report(Y_test, y_predicted, target_names=["Negativo", "Positivo"]))

## K-fold Cross Validation

In [ ]:
kCrossValidation(knc, featureSelection_df, labels)

# 6. Artificial Neural Network

## 6.1 Ottimizzazione dei parametri della rete neurale

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(featureSelection_df, labels, test_size=0.2, random_state=random_seed)

In [ ]:
mlp = MLPClassifier(max_iter=200, random_state = random_seed)

### Paramatri su cui viene effettuata la ricerca

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(200,150,100)],
    'activation': ['relu', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.005, 0.0001],
    'learning_rate': ['constant'],
    }

### Ricerca dei parametri

In [ ]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, Y_train)


### Parametri migliori trovati

In [ ]:
print('Parametri migliori:\n', clf.best_params_)

In [ ]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

## 6.2 Artificial Neural Network: Classificazione

### Prendo i parametri migliori trovati

In [ ]:
best_params = clf.best_params_ 

hidden_layer_sizes_found = best_params.get('hidden_layer_sizes')
activation_found = best_params.get('activation')
solver_found = best_params.get('solver')
alpha_found = best_params.get('alpha')
learning_rate_found = best_params.get('learning_rate')

### Task di classificazione

In [ ]:
mlp = MLPClassifier(max_iter=200, 
                    random_state = random_seed,
                    hidden_layer_sizes = hidden_layer_sizes_found,
                    activation = activation_found,
                    solver = solver_found,
                    alpha = alpha_found,
                    learning_rate = learning_rate_found
                   )

mlp.fit(X_train, Y_train)
y_predicted = mlp.predict(X_test)

In [ ]:
print("Accuracy score: ", accuracy_score(Y_test, y_predicted))
print(classification_report(Y_test, y_predicted, target_names=["Negativo", "Positivo"]))

# 7. Support Vector Machine

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(featureSelection_df, labels, test_size=0.2, random_state=random_seed)

## Valori su cui ho ottimizzato il classificatore SVC

In [ ]:
# valori su cui ho ottimizzato il classificatore SVC

# kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
# C_values = [0.5,1,10,100]
# gamma_values = ['scale', 1, 0.1, 0.01, 0.001, 0.0001]

# parametri migliori
kernel_found = 'linear'
C_found = 0.5
gamma_found = 0.0001

### Task di classificazione

In [ ]:
svm = SVC(random_state=random_seed, kernel = kernel_found, C = C_found, gamma = gamma_found)
svm.fit(X_train, Y_train)
y_predicted = svm.predict(X_test)

In [ ]:
print("Accuracy score: ", accuracy_score(Y_test, y_predicted))
print(classification_report(Y_test, y_predicted, target_names=["Negativo", "Positivo"]))

In [ ]:
plot_confusion_matrix(svm,
                     X_test,
                     Y_test,
                     values_format='d',
                     display_labels=["Negativo", "Positivo"])